In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sys
import os

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.features import preprocessing

In [2]:
# Bypass issues with invoking notebook with server arguments
sys.argv.clear()
sys.argv.append("")

In [3]:
%run utils/attention_graph.py
%run utils/mlflow_query.py
%run utils/loading.py
%run utils/comparison.py
%run utils/ranks.py

In [5]:
mlflow_helper = MlflowHelper(pkl_file=Path("mlflow_run_df.pkl"))
#mlflow_helper.query_runs(pkl_file=Path("mlflow_run_df.pkl"))

Initialized with 82 MLFlow runs from pkl


In [ ]:
sequences = pd.read_pickle("../data/sequences_df.pkl")
sequences

In [22]:
huawei_config = preprocessing.HuaweiPreprocessorConfig()
huawei_config.aggregated_log_file = Path("../data/logs_aggregated_concurrent.csv")
huawei_config.traces_root_directory = Path("../data/concurrent_data/traces/")
huawei_config.final_log_file = Path("../data/huawei.pkl")
huawei_config.remove_dates_from_payload = True

sequence_preprocessor = preprocessing.ConcurrentAggregatedLogsPreprocessor(huawei_config)
#huawei_full_data = sequence_preprocessor.load_full_data()

In [ ]:
huawei_full_data.columns

# Manually step through program flow

In [24]:
sequences_df = sequence_preprocessor._load_log_only_data()
sequences_df["grouper"] = 1
sequence_preprocessor._aggregate_per(sequences_df, aggregation_column="grouper")
sequences_df

Generating DRAIN clusters from log_df: 100%|██████████| 24/24 [00:00<00:00, 23098.51it/s]
/home/vincenzo/code/Domain-Guided-Monitoring/src/features/preprocessing/huawei.py:178: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  merged_df["all_events"] = merged_df[self.relevant_columns].values.tolist()


,Hostname,log_level,programname,python_module,http_status,http_method,@timestamp,Payload,http_url,fine_log_cluster_template,...,coarse_log_cluster_template,coarse_log_cluster_path,medium_log_cluster_template,medium_log_cluster_path,timestamp,url_cluster_template,url_cluster_path,grouper,all_events,attributes
0,Hostname#wally113,log_level#info,programname#neutron-openvswitch-agent,python_module#neutron.agent.securitygroups_rpc,,,2019-11-25T15:01:49.683000000+01:00,Security group rule updated ['78c81907-a991-41...,,[fine_log_cluster_template#security group rule...,...,[coarse_log_cluster_template#security group ru...,5 security group rule,[medium_log_cluster_template#security group ru...,5 security group rule updated *,2019-11-25 15:01:49.683000+01:00,,NaN,1,[fine_log_cluster_template#security group rule...,"[Hostname#wally113, , python_module#neutron.ag..."
1,Hostname#wally113,log_level#info,programname#neutron-openvswitch-agent,python_module#neutron.agent.securitygroups_rpc,,,2019-11-25T15:01:49.877000000+01:00,Security group rule updated ['78c81907-a991-41...,,[fine_log_cluster_template#security group rule...,...,[coarse_log_cluster_template#security group ru...,5 security group rule,[medium_log_cluster_template#security group ru...,5 security group rule updated *,2019-11-25 15:01:49.877000+01:00,,NaN,1,[fine_log_cluster_template#security group rule...,"[Hostname#wally113, , python_module#neutron.ag..."
2,Hostname#wally113,log_level#warning,programname#keystone,python_module#keystone.server.flask.application,,,2019-11-25T15:01:52.744000000+01:00,Could not find domain: Default.: DomainNotFoun...,,[fine_log_cluster_template#could not find doma...,...,[coarse_log_cluster_template#could not find do...,11 could not find,[medium_log_cluster_template#could not find do...,11 could not find domain default domainnotfound,2019-11-25 15:01:52.744000+01:00,,NaN,1,[fine_log_cluster_template#could not find doma...,"[Hostname#wally113, , python_module#keystone.s..."
3,Hostname#wally113,log_level#info,programname#glance-api,python_module#eventlet.wsgi.server,,,2019-11-25T15:01:48.510000000+01:00,"172.17.0.2 - - ""GET /v2/schemas/image HTTP/1....",,[fine_log_cluster_template#get v2 schemas imag...,...,[coarse_log_cluster_template#get v2 schemas im...,9 get v2 schemas,[medium_log_cluster_template#get v2 schemas im...,9 get v2 schemas image http *,2019-11-25 15:01:48.510000+01:00,,NaN,1,[fine_log_cluster_template#get v2 schemas imag...,"[Hostname#wally113, , python_module#eventlet.w..."
4,Hostname#wally113,log_level#info,programname#nova-api,python_module#nova.api.openstack,,,2019-11-25T15:01:56.520000000+01:00,http://130.149.249.123:8774/v2.1/dbf4ab7d6e844...,,[fine_log_cluster_template#http v21 dbf4ab7d6e...,...,[coarse_log_cluster_template#http v21 dbf4ab7d...,8 http v21 *,[medium_log_cluster_template#http v21 dbf4ab7d...,8 http v21 * servers returned with,2019-11-25 15:01:56.520000+01:00,,NaN,1,[fine_log_cluster_template#http v21 dbf4ab7d6e...,"[Hostname#wally113, , python_module#nova.api.o..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177,Hostname#wally122,log_level#info,programname#neutron-openvswitch-agent,python_module#neutron.agent.securitygroups_rpc,,,2019-11-25T15:02:08.342000000+01:00,Security group rule updated ['4674feac-20f6-4a...,,[fine_log_cluster_template#security group rule...,...,[coarse_log_cluster_template#security group ru...,5 security group rule,[medium_log_cluster_template#security group ru...,5 security group rule updated *,2019-11-25 15:02:08.342000+01:00,,NaN,1,[fine_log_cluster_template#security group rule...,"[Hostname#wally122, , python_module#neutron.ag..."
178,Hostname#wally122,log_level#info,programname#neutron-openvswitch-agent,python_module#neutron.agent.securitygroups_rpc,,,2019-11-25T15:02:08.674000000+01:00,Security group rule updated ['4674feac-20f6-4a...,,[fine_log_cluster_template#security group rule...,...,[coarse_log_cluster_template#security group ru...,5

In [26]:
sequences_df.head()

,Hostname,log_level,programname,python_module,http_status,http_method,@timestamp,Payload,http_url,fine_log_cluster_template,fine_log_cluster_path,coarse_log_cluster_template,coarse_log_cluster_path,medium_log_cluster_template,medium_log_cluster_path,timestamp,url_cluster_template,url_cluster_path
0,wally113,INFO,neutron-openvswitch-agent,neutron.agent.securitygroups_rpc,,,2019-11-25T15:01:49.683000000+01:00,Security group rule updated ['78c81907-a991-41...,,security group rule updated *,5 security group rule updated *,security group rule updated *,5 security group rule,security group rule updated *,5 security group rule updated *,2019-11-25 15:01:49.683000+01:00,,NaN
1,wally113,INFO,neutron-openvswitch-agent,neutron.agent.securitygroups_rpc,,,2019-11-25T15:01:49.877000000+01:00,Security group rule updated ['78c81907-a991-41...,,security group rule updated *,5 security group rule updated *,security group rule updated *,5 security group rule,security group rule updated *,5 security group rule updated *,2019-11-25 15:01:49.877000+01:00,,NaN
2,wally113,WARNING,keystone,keystone.server.flask.application,,,2019-11-25T15:01:52.744000000+01:00,Could not find domain: Default.: DomainNotFoun...,,could not find domain default domainnotfound c...,11 could not find domain default domainnotfoun...,could not find domain default domainnotfound c...,11 could not find,could not find domain default domainnotfound c...,11 could not find domain default domainnotfound,2019-11-25 15:01:52.744000+01:00,,NaN
3,wally113,INFO,glance-api,eventlet.wsgi.server,,,2019-11-25T15:01:48.510000000+01:00,"172.17.0.2 - - ""GET /v2/schemas/image HTTP/1....",,get v2 schemas image http 11 200 5562 *,9 get v2 schemas image http * * * *,get v2 schemas image http 11 200 5562 *,9 get v2 schemas,get v2 schemas image http 11 200 5562 *,9 get v2 schemas image http *,2019-11-25 15:01:48.510000+01:00,,NaN
4,wally113,INFO,nova-api,nova.api.openstack,,,2019-11-25T15:01:56.520000000+01:00,http://130.149.249.123:8774/v2.1/dbf4ab7d6e844...,,http v21 dbf4ab7d6e84449e93e02f305534200b serv...,8 http v21 * servers returned with http *,http v21 dbf4ab7d6e84449e93e02f305534200b serv...,8 http v21 *,http v21 dbf4ab7d6e84449e93e02f305534200b serv...,8 http v21 * servers returned with,2019-11-25 15:01:56.520000+01:00,,NaN


In [6]:
read_log = sequence_preprocessor._read_log_df()
read_log.head()


Generating DRAIN clusters from log_df: 100%|██████████| 163/163 [00:00<00:00, 14465.57it/s]


,Hostname,log_level,programname,python_module,http_status,http_method,@timestamp,Payload,http_url,fine_log_cluster_template,fine_log_cluster_path,coarse_log_cluster_template,coarse_log_cluster_path,medium_log_cluster_template,medium_log_cluster_path,timestamp
0,wally113,INFO,neutron-openvswitch-agent,neutron.agent.securitygroups_rpc,,,2019-11-25T15:01:49.683000000+01:00,Security group rule updated ['78c81907-a991-41...,,security group rule updated *,5 security group rule updated *,security group rule updated *,5 security group rule,security group rule updated *,5 security group rule updated *,2019-11-25 15:01:49.683000+01:00
1,wally113,INFO,neutron-openvswitch-agent,neutron.agent.securitygroups_rpc,,,2019-11-25T15:01:49.877000000+01:00,Security group rule updated ['78c81907-a991-41...,,security group rule updated *,5 security group rule updated *,security group rule updated *,5 security group rule,security group rule updated *,5 security group rule updated *,2019-11-25 15:01:49.877000+01:00
2,wally113,WARNING,keystone,keystone.server.flask.application,,,2019-11-25T15:01:52.744000000+01:00,Could not find domain: Default.: DomainNotFoun...,,could not find domain default domainnotfound c...,11 could not find domain default domainnotfoun...,could not find domain default domainnotfound c...,11 could not find,could not find domain default domainnotfound c...,11 could not find domain default domainnotfound,2019-11-25 15:01:52.744000+01:00
3,wally113,INFO,glance-api,eventlet.wsgi.server,,,2019-11-25T15:01:48.510000000+01:00,"172.17.0.2 - - ""GET /v2/schemas/image HTTP/1....",,get v2 schemas image http 11 200 5562 *,9 get v2 schemas image http * * * *,get v2 schemas image http 11 200 5562 *,9 get v2 schemas,get v2 schemas image http 11 200 5562 *,9 get v2 schemas image http *,2019-11-25 15:01:48.510000+01:00
4,wally113,INFO,nova-api,nova.api.openstack,,,2019-11-25T15:01:56.520000000+01:00,http://130.149.249.123:8774/v2.1/dbf4ab7d6e844...,,http v21 dbf4ab7d6e84449e93e02f305534200b serv...,8 http v21 * servers returned with http *,http v21 dbf4ab7d6e84449e93e02f305534200b serv...,8 http v21 *,http v21 dbf4ab7d6e84449e93e02f305534200b serv...,8 http v21 * servers returned with,2019-11-25 15:01:56.520000+01:00


In [28]:
data_df = sequence_preprocessor.load_data()

Generating DRAIN clusters from log_df: 100%|██████████| 24/24 [00:00<00:00, 12553.10it/s]
/home/vincenzo/code/Domain-Guided-Monitoring/src/features/preprocessing/huawei.py:178: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  merged_df["all_events"] = merged_df[self.relevant_columns].values.tolist()


KeyError: 0

In [31]:
list(data_df["fine_log_cluster_template"])

[[['fine_log_cluster_template#get resource providers 45bac5db7b404922ae54fe747131258d allocations http 11 200 57 18871 novacompute keystoneauth1 3131 pythonrequests 2210 cpython 2715'],
  ['fine_log_cluster_template#get http 11 300 270 * rally keystoneauth1 3110 pythonrequests 2210 cpython 2712'],
  ['fine_log_cluster_template#post v3 auth tokens http 11 201 * * rally keystoneauth1 3110 pythonrequests 2210 cpython 2712'],
  ['fine_log_cluster_template#post v3 auth tokens http 11 201 * * rally keystoneauth1 3110 pythonrequests 2210 cpython 2712'],
  ['fine_log_cluster_template#post v3 projects http 11 201 304 * pythonkeystoneclient'],
  ['fine_log_cluster_template#get v3 domains name default http 11 200 293 * pythonkeystoneclient'],
  ['fine_log_cluster_template#post v3 auth tokens http 11 201 * * rally keystoneauth1 3110 pythonrequests 2210 cpython 2712'],
  ['fine_log_cluster_template#put v3 projects * users * roles 45b34dafb0d745a8873f482c0c32ff15 http 11 204 * pythonkeystoneclient']

# Dataset exploration

In [19]:
dataset_df = pd.read_csv(Path("../data/logs_aggregated_concurrent.csv")).fillna("").astype(str).replace(np.nan, "", regex=True)
dataset_df

,_id,_index,_score,_type,Hostname,user_id,project_domain,Timestamp,@timestamp,log_level,...,http_method,http_version,http_url,chunk,next_retry_seconds,error,retry_time,message,chunk_id,worker
0,AW6i3n4JtOpH-3YEBpA7,flog-2019.11.25,1.0,fluentd,wally113,2ac494fefe7848abb12faa65b74cd990,-,2019-11-25 15:01:49.683,2019-11-25T15:01:49.683000000+01:00,INFO,...,,,,,,,,,,
1,AW6i3n4JtOpH-3YEBpBB,flog-2019.11.25,1.0,fluentd,wally113,2537c5418d8a45d6abfcade96cfd838a,-,2019-11-25 15:01:49.877,2019-11-25T15:01:49.877000000+01:00,INFO,...,,,,,,,,,,
2,AW6i3n4JtOpH-3YEBpBD,flog-2019.11.25,1.0,fluentd,wally113,2537c5418d8a45d6abfcade96cfd838a,default,2019-11-25 15:01:52.744,2019-11-25T15:01:52.744000000+01:00,WARNING,...,,,,,,,,,,
3,AW6i3n4JtOpH-3YEBpBL,flog-2019.11.25,1.0,fluentd,wally113,2ac494fefe7848abb12faa65b74cd990,default,2019-11-25 15:01:48.510,2019-11-25T15:01:48.510000000+01:00,INFO,...,,,,,,,,,,
4,AW6i3n4JtOpH-3YEBpBR,flog-2019.11.25,1.0,fluentd,wally113,,,2019-11-25 15:01:56.520,2019-11-25T15:01:56.520000000+01:00,INFO,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177,AW6i3tvLtOpH-3YEBpHM,flog-2019.11.25,1.0,fluentd,wally122,6880eaed9e8f4d2c9ba19524b68c2e4a,-,2019-11-25 15:02:08.342,2019-11-25T15:02:08.342000000+01:00,INFO,...,,,,,,,,,,
178,AW6i3tvLtOpH-3YEBpHN,flog-2019.11.25,1.0,fluentd,wally122,2537c5418d8a45d6abfcade96cfd838a,-,2019-11-25 15:02:08.674,2019-11-25T15:02:08.674000000+01:00,INFO,...,,,,,,,,,,
179,AW6i3tvLtOpH-3YEBpHT,flog-2019.11.25,1.0,fluentd,wally122,6f6a41b8a1c642ddaaab4ef3c138dfb4,-,2019-11-25 15:02:09.130,2019-11-25T15:02:09.130000000+01:00,INFO,...,,,,,,,,,,
180,AW6i3tsZtOpH-3YEBpGq,flog-2019.11.25,1.0,fluentd,wally117,6880eaed9e8f4d2c9ba19524b68c2e4a,-,2019-11-25 15:02:08.341,2019-11-25T15:02:08.341000000+01:00,INFO,...,,,,,,,,,,


In [4]:
from src.features.preprocessing import spell
LOG_FORMAT = '<Date> <Time> <Pid> <Level> <Component>: <Content>'
LOG_FORMAT_HUAWEI = '<Hostname> <log_level> <Payload> <programname> <python_module> <http_status> <http_method> <http_url>'

In [5]:
parser = spell.LogParser(indir="../data/", log_format=LOG_FORMAT, csv=False, content="Content")

In [6]:
parser.parse("test_data.log")

[2023-01-23 20:15:46,059][INFO]: Parsing file: ../data/test_data.log
[2023-01-23 20:15:46,061][INFO]: load_data() finished!
[2023-01-23 20:15:46,063][INFO]: Load objects done, lastestLineId: 1110
[2023-01-23 20:15:46,064][INFO]: Processed 100.0% of log lines.
[2023-01-23 20:15:46,066][INFO]: Output parse file
[2023-01-23 20:15:46,068][INFO]: rootNodePath: ./result/rootNode.pkl
[2023-01-23 20:15:46,070][INFO]: logCluLPath: ./result/logCluL.pkl
[2023-01-23 20:15:46,071][INFO]: Store objects done.
[2023-01-23 20:15:46,071][INFO]: Parsing done. [Time taken: 0:00:00.012377]


In [20]:
spell_huawei_parser = spell.LogParser(indir="../data/", log_format=LOG_FORMAT_HUAWEI, text_max_length=4096, keep_para=False, tau=0.7)

In [21]:
spell_huawei_parser.parse("logs_aggregated_concurrent.csv")

[2023-01-23 22:31:54,113][INFO]: Parsing file: ../data/logs_aggregated_concurrent.csv
[2023-01-23 22:31:54,116][INFO]: load_data() finished!
[2023-01-23 22:31:54,119][INFO]: Load objects done, lastestLineId: 182
[2023-01-23 22:31:54,135][INFO]: Processed 100.0% of log lines.
[2023-01-23 22:31:54,137][INFO]: Output parse file
[2023-01-23 22:31:54,141][INFO]: rootNodePath: ./result/rootNode.pkl
[2023-01-23 22:31:54,144][INFO]: logCluLPath: ./result/logCluL.pkl
[2023-01-23 22:31:54,145][INFO]: Store objects done.
[2023-01-23 22:31:54,145][INFO]: Parsing done. [Time taken: 0:00:00.032249]


In [12]:
logname = "logs_aggregated_concurrent.csv"
headers, regex = spell_huawei_parser.generate_logformat_regex(LOG_FORMAT)
parser_df = spell_huawei_parser.log_to_dataframe("../data/logs_aggregated_concurrent.csv", regex, headers, LOG_FORMAT)

In [13]:
parser_df

,LineId,_id,_index,_score,_type,Hostname,user_id,project_domain,Timestamp,@timestamp,...,http_method,http_version,http_url,chunk,next_retry_seconds,error,retry_time,message,chunk_id,worker
0,1,AW6i3n4JtOpH-3YEBpA7,flog-2019.11.25,1.0,fluentd,wally113,2ac494fefe7848abb12faa65b74cd990,-,2019-11-25 15:01:49.683,2019-11-25T15:01:49.683000000+01:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,AW6i3n4JtOpH-3YEBpBB,flog-2019.11.25,1.0,fluentd,wally113,2537c5418d8a45d6abfcade96cfd838a,-,2019-11-25 15:01:49.877,2019-11-25T15:01:49.877000000+01:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,AW6i3n4JtOpH-3YEBpBD,flog-2019.11.25,1.0,fluentd,wally113,2537c5418d8a45d6abfcade96cfd838a,default,2019-11-25 15:01:52.744,2019-11-25T15:01:52.744000000+01:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,AW6i3n4JtOpH-3YEBpBL,flog-2019.11.25,1.0,fluentd,wally113,2ac494fefe7848abb12faa65b74cd990,default,2019-11-25 15:01:48.510,2019-11-25T15:01:48.510000000+01:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,AW6i3n4JtOpH-3YEBpBR,flog-2019.11.25,1.0,fluentd,wally113,NaN,NaN,2019-11-25 15:01:56.520,2019-11-25T15:01:56.520000000+01:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177,178,AW6i3tvLtOpH-3YEBpHM,flog-2019.11.25,1.0,fluentd,wally122,6880eaed9e8f4d2c9ba19524b68c2e4a,-,2019-11-25 15:02:08.342,2019-11-25T15:02:08.342000000+01:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
178,179,AW6i3tvLtOpH-3YEBpHN,flog-2019.11.25,1.0,fluentd,wally122,2537c5418d8a45d6abfcade96cfd838a,-,2019-11-25 15:02:08.674,2019-11-25T15:02:08.674000000+01:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
179,180,AW6i3tvLtOpH-3YEBpHT,flog-2019.11.25,1.0,fluentd,wally122,6f6a41b8a1c642ddaaab4ef3c138dfb4,-,2019-11-25 15:02:09.130,2019-11-25T15:02:09.130000000+01:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
180,181,AW6i3tsZtOpH-3YEBpGq,flog-2019.11.25,1.0,fluentd,wally117,6880eaed9e8f4d2c9ba19524b68c2e4a,-,2019-11-25 15:02:08.341,2019-11-25T15:02:08.341000000+01:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
dataset_df.insert(0, 'LineId', None)
dataset_df['LineId'] = [i + 1 for i in range(len(dataset_df.index))]
dataset_df

,LineId,_id,_index,_score,_type,Hostname,user_id,project_domain,Timestamp,@timestamp,...,http_method,http_version,http_url,chunk,next_retry_seconds,error,retry_time,message,chunk_id,worker
0,1,AW6i3n4JtOpH-3YEBpA7,flog-2019.11.25,1.0,fluentd,wally113,2ac494fefe7848abb12faa65b74cd990,-,2019-11-25 15:01:49.683,2019-11-25T15:01:49.683000000+01:00,...,,,,,,,,,,
1,2,AW6i3n4JtOpH-3YEBpBB,flog-2019.11.25,1.0,fluentd,wally113,2537c5418d8a45d6abfcade96cfd838a,-,2019-11-25 15:01:49.877,2019-11-25T15:01:49.877000000+01:00,...,,,,,,,,,,
2,3,AW6i3n4JtOpH-3YEBpBD,flog-2019.11.25,1.0,fluentd,wally113,2537c5418d8a45d6abfcade96cfd838a,default,2019-11-25 15:01:52.744,2019-11-25T15:01:52.744000000+01:00,...,,,,,,,,,,
3,4,AW6i3n4JtOpH-3YEBpBL,flog-2019.11.25,1.0,fluentd,wally113,2ac494fefe7848abb12faa65b74cd990,default,2019-11-25 15:01:48.510,2019-11-25T15:01:48.510000000+01:00,...,,,,,,,,,,
4,5,AW6i3n4JtOpH-3YEBpBR,flog-2019.11.25,1.0,fluentd,wally113,,,2019-11-25 15:01:56.520,2019-11-25T15:01:56.520000000+01:00,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177,178,AW6i3tvLtOpH-3YEBpHM,flog-2019.11.25,1.0,fluentd,wally122,6880eaed9e8f4d2c9ba19524b68c2e4a,-,2019-11-25 15:02:08.342,2019-11-25T15:02:08.342000000+01:00,...,,,,,,,,,,
178,179,AW6i3tvLtOpH-3YEBpHN,flog-2019.11.25,1.0,fluentd,wally122,2537c5418d8a45d6abfcade96cfd838a,-,2019-11-25 15:02:08.674,2019-11-25T15:02:08.674000000+01:00,...,,,,,,,,,,
179,180,AW6i3tvLtOpH-3YEBpHT,flog-2019.11.25,1.0,fluentd,wally122,6f6a41b8a1c642ddaaab4ef3c138dfb4,-,2019-11-25 15:02:09.130,2019-11-25T15:02:09.130000000+01:00,...,,,,,,,,,,
180,181,AW6i3tsZtOpH-3YEBpGq,flog-2019.11.25,1.0,fluentd,wally117,6880eaed9e8f4d2c9ba19524b68c2e4a,-,2019-11-25 15:02:08.341,2019-11-25T15:02:08.341000000+01:00,...,,,,,,,,,,


In [7]:
parser.logname = "test_data.log"
parser.load_data()

In [8]:
parser.df_log

,LineId,Date,Time,Pid,Level,Component,Content
0,1,081109,203518,143,INFO,dfs.DataNode$DataXceiver,Receiving block blk_-1608999687919862906 src: ...
1,2,081109,203518,35,INFO,dfs.FSNamesystem,BLOCK* NameSystem.allocateBlock: /mnt/hadoop/m...
2,3,081109,203519,143,INFO,dfs.DataNode$DataXceiver,Receiving block blk_-1608999687919862906 src: ...
